In [28]:
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import os
import re
import glob
import pandas as pd
import random
import imgaug
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from skimage.transform import rescale, resize, downscale_local_mean
from os import path, mkdir
import tensorflow as tf 
import numpy as np 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import metrics
from tensorflow.keras.utils import*

In [29]:
train_data = pd.read_csv('/home/bmi/kits19/train_data.csv')
test_data = pd.read_csv('/home/bmi/kits19/test_data.csv')
valid_data = pd.read_csv('/home/bmi/kits19/valid_data.csv')

In [30]:
image_path = '../sliced_img/'
mask_path = '../sliced_mask/'
train_data = [image_path + i for i in pd.read_csv('/home/bmi/kits19/train_data.csv')['Image_path']]
# print (train_data)
# for i in train_data['Image_path']:
#     print (image_path+i)

In [31]:
print (len(train_data), len(test_data), len(valid_data))

31808 3600 9764


In [32]:
# m = len(csv_file)
# print(m,np.int(np.ceil(m*0.7)))
# tarin_df = pd.DataFrame()
# valid_df = pd.DataFrame()
# test_df = pd.DataFrame()
# train_df = csv_file.iloc[:,:np.int(np.ceil(m*0.7))]
# print (len(train_df))
# test_df = csv_file.iloc[:,np.int(np.ceil(m*0.9)):]
# print(len(test_df))
# valid_df = csv_file.iloc[:,np.int(np.ceil(m*0.7)):np.int(np.ceil(m*0.9))]
# print (len(valid_df))
# train_df.to_csv('../train_csv_.csv')
# valid_df.to_csv('../valid_csv_file.csv')
# test_df.to_csv('../test_csv_file.csv')

In [33]:
def imshow(*args,**kwargs):
    """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
    Usage:
    imshow(img1, title="myPlot")
    imshow(img1,img2, title=['title1','title2'])
    imshow(img1,img2, cmap='hot')
    imshow(img1,img2,cmap=['gray','Blues']) """
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    axis_off = kwargs.get('axis_off','')
    if len(args)==0:
        raise ValueError("No images given to imshow")
    elif len(args)==1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n=len(args)
        if type(cmap)==str:
            cmap = [cmap]*n
        if type(title)==str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
            if axis_off: 
              plt.axis('off')  
    plt.show()

In [46]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,image_path, mask_path, data_dir, image_size=(512,512), batch_size=32, n_classes=3, n_channels=1,
                  shuffle=True, transform=None):
        'Initialization'
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.data_dir = data_dir

        self.images = sorted(image_path + i for i in pd.read_csv(data_dir)['Image_path'])
        self.masks = sorted(mask_path + i for i in pd.read_csv(data_dir)['Image_path'])

        self.image_size = image_size
        self.n_channels = n_channels
       
        self.transform = transform
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        image_batch_list = self.images[index*self.batch_size:(index+1)*self.batch_size]
        mask_batch_list = self.masks[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(image_batch_list, mask_batch_list)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            mapIndexPosition = list(zip(self.images, self.masks))
            random.shuffle(mapIndexPosition)
            self.images, self.masks = zip(*mapIndexPosition)

    def _get_one_hot(self, targets):
        res = np.eye(self.n_classes)[np.array(targets).reshape(-1)]
        return res.reshape(list(targets.shape)+[self.n_classes])

    def _preprocess_image(self, image):

        # Normalize Image
        image =image
        # image = image * (1.0 / 255)  
        # #  ImageNet Standardization
        # imagenet_mean = np.array([0.485, 0.456, 0.406])
        # imagenet_std = np.array([0.229, 0.224, 0.225])
        # image = (image - imagenet_mean) / imagenet_std 
        return image
    
    def _augmentation(self, image, mask):
        # Augmenters that are safe to apply to masks
        # Some, such as Affine, have settings that make them unsafe, so always
        # test your augmentation on masks
        MASK_AUGMENTERS = ["Sequential", "SomeOf", "OneOf", "Sometimes",
                           "Fliplr", "Flipud", "CropAndPad",
                           "Affine", "PiecewiseAffine"]

        def hook(images, augmenter, parents, default):
            """Determines which augmenters to apply to masks."""
            return augmenter.__class__.__name__ in MASK_AUGMENTERS

        # Store shapes before augmentation to compare
        image_shape = image.shape
        mask_shape = mask.shape
        # Make augmenters deterministic to apply similarly to images and masks
        det = self.transform.to_deterministic()
        image = det.augment_image(image)
        # Change mask to np.uint8 because imgaug doesn't support np.bool
        mask = det.augment_image(mask.astype(np.uint8),
                                 hooks=imgaug.HooksImages(activator=hook))
        # Verify that shapes didn't change
        assert image.shape == image_shape, "Augmentation shouldn't change image size"
        assert mask.shape == mask_shape, "Augmentation shouldn't change mask size"
        # Change mask back to bool
        # label = label.astype(np.bool)
        return image, mask
  
    def __data_generation(self, image_batch_list, mask_batch_list):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.image_size, self.n_channels))
        y = np.empty((self.batch_size, *self.image_size, self.n_classes))

        # Generate data
        for i, image_mask_path in enumerate(zip(image_batch_list, mask_batch_list)):
            image = sitk.GetArrayFromImage(sitk.ReadImage(image_mask_path[0]))
            image = resize(image,(512,512,1))
            image = np.asarray(image)
            mask = sitk.GetArrayFromImage(sitk.ReadImage(image_mask_path[1]))
#             mask = resize(mask,(512,512,1))
            mask = np.asarray(mask)
#             image = np.asarray(resize((sitk.GetArrayFromImage(sitk.ReadImage(image_mask_path[0])))),(512,512,1))
#             mask =  np.asarray(resize((sitk.GetArrayFromImage(sitk.ReadImage(image_mask_path[1])))),(512,512,1))

            image = self._preprocess_image(image)
            mask = self._get_one_hot(mask)
            if self.transform:
                image, mask = self._augmentation(image, mask)
            X[i,] = image
            y[i,] = mask
        return X, y

In [47]:
# Training Data Configuration
# Data Path
image_path = '../sliced_img/'
mask_path = '../sliced_mask/'
train_data_dir = '../train_data.csv'
valid_data_dir = '../valid_data.csv'
augmentation = iaa.SomeOf((0, 3), 
            [
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
                iaa.Noop(),
                iaa.OneOf([iaa.Affine(rotate=90),
                           iaa.Affine(rotate=180),
                           iaa.Affine(rotate=270)]),
                iaa.GaussianBlur(sigma=(0.0, 0.5)),
            ])
# Parameters
train_transform_params = {'image_size': (512,512),
                          'batch_size': 32,
                          'n_classes': 3,
                          'n_channels': 1,
                          'shuffle': True,                          
                          'transform': augmentation
                         }

valid_transform_params = {'image_size': (512,512),
                          'batch_size': 32,
                          'n_classes': 3,
                          'n_channels': 1,
                          'shuffle': True,                         
                          'transform': None
                         }
# Generators
training_generator = DataGenerator(image_path, mask_path, train_data_dir, **train_transform_params)
validation_generator = DataGenerator(image_path, mask_path, valid_data_dir, **valid_transform_params)

# Enable Test Code
print (training_generator.__len__(), validation_generator.__len__())
for X, y in training_generator:
    print (X.shape, y.shape)
#     imshow(X[0], y[0][:,:,1])

994 305
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)
(32, 512, 512, 1) (32, 512, 512, 3)


SystemError: <built-in function ReadImage> returned a result with an error set

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [17]:
def generate_data(image_path, mask_path, csv_directory, n_class, batch_size ,augmentation = None):
    """Replaces Keras' native ImageDataGenerator."""
    i = 0
    image_fps = pd.read_csv(csv_directory)
    image_fps_train = image_fps['Image_path']
 
    while True:
        image_batch =[]
        label_batch =[]
       
        for b in range(batch_size):          
            if i == len(image_fps_train):
                i = 0
                random.shuffle(image_fps_train)
            sample = image_fps_train[i]
            i += 1
            # print (sample)
            img_path = image_path + '/' + sample
            gt_path = mask_path + '/' + sample            
            ds = sitk.ReadImage(img_path)
            img = sitk.GetArrayFromImage(ds)
            # img = resize(img, (512,512,1))
            # image[:,:,b] = img
            gt = sitk.ReadImage(gt_path)
            gt = sitk.GetArrayFromImage(gt)
            # gt = resize(gt, (512,512,1))
            # label[:,:,b] = gt
            label = get_one_hot(gt,n_class)


            # Data Augmentation
            # if augmentation:
            #     import imgaug
            #     # Augmenters that are safe to apply to masks
            #     # Some, such as Affine, have settings that make them unsafe, so always
            #     # test your augmentation on masks
            #     MASK_AUGMENTERS = ["Sequential", "SomeOf", "OneOf", "Sometimes",
            #                        "Fliplr", "Flipud", "CropAndPad",
            #                        "Affine", "PiecewiseAffine"]

            #     def hook(images, augmenter, parents, default):
            #         """Determines which augmenters to apply to masks."""
            #         return augmenter.__class__.__name__ in MASK_AUGMENTERS

            #     # Store shapes before augmentation to compare
            #     image_shape = image.shape
            #     label_shape = label.shape
            #     # Make augmenters deterministic to apply similarly to images and masks
            #     det = augmentation.to_deterministic()
            #     image = det.augment_image(image)
            #     # Change mask to np.uint8 because imgaug doesn't support np.bool
            #     label = det.augment_image(label.astype(np.uint8),
            #                              hooks=imgaug.HooksImages(activator=hook))
            #     # Verify that shapes didn't change
            #     assert image.shape == image_shape, "Augmentation shouldn't change image size"
            #     assert label.shape == label_shape, "Augmentation shouldn't change mask size"
            #     # Change mask back to bool
            #     # label = label.astype(np.bool)
            # image_batch.append(image)
            # label_batch.append(label)
            image_batch.append(img)
            label_batch.append(label)
        yield np.array(image_batch), np.array(label_batch)

In [18]:
batch_size = 4
n_class = 3
image_path = '/home/bmi/kits19/sliced_img'
mask_path = '/home/bmi/kits19/sliced_mask'
csv_directory = '/home/bmi/kits19/train_data.csv'

#     augmentation = iaa.SomeOf((0, 3), [
#         iaa.Fliplr(0.4),
#         iaa.Flipud(0.2),
#         iaa.Noop(),
#         iaa.OneOf([iaa.Affine(rotate=90),
#                    iaa.Affine(rotate=180),
#                    iaa.Affine(rotate=270)]),
#         iaa.GaussianBlur(sigma=(0.0, 0.5)),
#         iaa.Noop()
#     ])
train_generator = generate_data(image_path, mask_path, csv_directory,n_class, batch_size,augmentation = None)

for X, y in train_generator:
    print (X.shape, y.shape)
    

(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512, 512) (4, 512, 512, 3)
(4, 512,

SystemError: <built-in function ReadImage> returned a result with an error set